In [1]:
__depends__=[]
__dest__= ["../results/ebtel_varying_tau_results.pickle",
           "../results/ebtel_varying_flux_limiter_results.pickle"
          ]

# Compute EBTEL Results
Run the single- and two-fluid EBTEL models for a variety of inputs. This will be the basis for the rest of our analysis.

First, import any needed modules.

In [6]:
import sys
import os
import subprocess
import pickle

import numpy as np

sys.path.append(os.path.join(os.environ['EXP_DIR'],'ebtelPlusPlus/rsp_toolkit/python'))
from xml_io import InputHandler,OutputHandler

Setup the base dictionary for all of the runs. We'll read in the base dictionary from the ebtel++ example configuration file.

In [7]:
ih = InputHandler(os.path.join(os.environ['EXP_DIR'],'ebtelPlusPlus','config','ebtel.example.cfg.xml'))
config_dict = ih.lookup_vars()

In [8]:
config_dict['use_adaptive_solver'] = False
config_dict['loop_length'] = 40.0e+8
config_dict['adaptive_solver_error'] = 1e-8
config_dict['calculate_dem'] = False
config_dict['total_time'] = 5000.0
config_dict['tau'] = 0.1
config_dict['use_c1_grav_correction'] = True
config_dict['use_c1_loss_correction'] = True
config_dict['c1_cond0'] = 6.0
config_dict['c1_rad0'] = 0.6
config_dict['heating']['background'] = 3.5e-5
config_dict['output_filename'] = '../results/_tmp_'

Next, construct a function that will make it easy to run all of the different EBTEL configurations.

In [9]:
def run_and_print(tau,h0,f,flux_opt,oh_inst):
    #create heating event
    oh_inst.output_dict['heating']['events'] = [
        {'event':{'magnitude':h0,'rise_start':0.0,'rise_end':tau/2.0,'decay_start':tau/2.0,'decay_end':tau}}
    ]
    #set heat flux options
    oh_inst.output_dict['saturation_limit'] = f
    oh_inst.output_dict['use_flux_limiting'] = flux_opt
    #single-fluid    
    oh_inst.output_dict['force_single_fluid'] = True
    oh_inst.output_dict['heating']['partition'] = 0.5
    oh_inst.print_to_xml()
    subprocess.call([os.path.join(os.environ['EXP_DIR'],'ebtelPlusPlus','bin','ebtel++.run'),
                     '-c',oh_inst.output_filename])
    #save parameters to list
    temp = np.loadtxt(oh_inst.output_dict['output_filename'])
    t,T,n = temp[:,0],temp[:,1],temp[:,3]
    #two-fluid
    #--electron heating
    oh_inst.output_dict['force_single_fluid'] = False
    oh_inst.output_dict['heating']['partition'] = 1.0
    oh_inst.print_to_xml()
    subprocess.call([os.path.join(os.environ['EXP_DIR'],'ebtelPlusPlus','bin','ebtel++.run'),
                     '-c',oh_inst.output_filename])
    temp = np.loadtxt(oh_inst.output_dict['output_filename'])
    te,Tee,Tei,ne= temp[:,0],temp[:,1],temp[:,2],temp[:,3]
    #--ion heating
    oh_inst.output_dict['force_single_fluid'] = False
    oh_inst.output_dict['heating']['partition'] = 0.0
    oh_inst.print_to_xml()
    subprocess.call([os.path.join(os.environ['EXP_DIR'],'ebtelPlusPlus','bin','ebtel++.run'),
                     '-c',oh_inst.output_filename])
    temp = np.loadtxt(oh_inst.output_dict['output_filename'])
    ti,Tie,Tii,ni = temp[:,0],temp[:,1],temp[:,2],temp[:,3]
    #return dictionary
    return {'t':t,'te':te,'ti':ti,'T':T,'Tee':Tee,'Tei':Tei,'Tie':Tie,'Tii':Tii,'n':n,'ne':ne,'ni':ni,
           'heat_flux_option':flux_opt}

Configure instances of the XML output handler for printing files.

In [10]:
oh = OutputHandler(config_dict['output_filename']+'.xml',config_dict)

Finally, run the model for varying pulse duration.

In [11]:
tau_h = [20,40,200,500]
tau_h_results = []
for t in tau_h:
    results = run_and_print(t,20.0/t,1.0,True,oh)
    results['loop_length'] = config_dict['loop_length']
    tau_h_results.append(results)

And then run the models for varying flux-limiter, $f$.

In [12]:
flux_lim = [{'f':1.0,'opt':True},{'f':0.53,'opt':True},{'f':1.0/6.0,'opt':True},{'f':0.1,'opt':True},
            {'f':1.0/30.0,'opt':True},{'f':1.0,'opt':False}]
flux_lim_results = []
for i in range(len(flux_lim)):
    results = run_and_print(200.0,0.1,flux_lim[i]['f'],flux_lim[i]['opt'],oh)
    results['loop_length'] = config_dict['loop_length']
    flux_lim_results.append(results)

Save both data structures to serialized files.

In [13]:
with open(__dest__[0],'wb') as f:
    pickle.dump(tau_h_results,f)
with open(__dest__[1],'wb') as f:
    pickle.dump(flux_lim_results,f)